<H2><CENTER> Interactive Data Visualization Capabilities of AIDA using TPC-H Data set</CENTER></H2>

In this workflow, we will demonstrate how AIDA is capable of providing sophistiated data visualization support even though it is a database resident framework. Here we will use the data set of TPC-H query 16 to plot the output over a world map, and an interactive widget to allow the user to select certain parameters of the query.

<HR>
Once we import AIDA components, we are good to go !

In [1]:
from aida.aida import *;

Let us start by establishing a connection to the database and get a connection object. This is syntactically very similar to any JDBC/ODBC based applications.

In [2]:
host='cerberus'; dbname='demo'; user='demo'; passwd='demo'; jobName='TPCHExplorerQ16'; port=55660;
dw = AIDA.connect(host, dbname, user, passwd, jobName, port);

We will express TPC-H Query 16 using AIDA's ORM syntax.

In [3]:
c = dw.customer; l = dw.lineitem; s = dw.supplier; n = dw.nation; 
r = dw.region.filter(Q('r_name', C('ASIA')));
o = dw.orders.filter(Q('o_orderdate', DATE('1994-01-01'), CMP.GTE), Q('o_orderdate', DATE('1995-01-01'), CMP.LT));

t = c.join(o, ('c_custkey',),('o_custkey',), COL.ALL, COL.ALL);
t = t.join(l, ('o_orderkey',), ('l_orderkey',), COL.ALL, COL.ALL);
t = t.join(s, ('l_suppkey', 'c_nationkey'), ('s_suppkey', 's_nationkey'), COL.ALL, COL.ALL);
t = t.join(n, ('s_nationkey',), ('n_nationkey',), COL.ALL, COL.ALL);
t = t.join(r, ('n_regionkey',), ('r_regionkey',), COL.ALL, COL.ALL);
t = t.project(('n_name', {F('l_extendedprice')*(1-F('l_discount')):'rev'}))
t = t.aggregate(('n_name', {SUM('rev'):'revenue'}), ('n_name',));
t = t.order('revenue#desc');

In [4]:
head(t);

      n_name       revenue
0  INDONESIA  5.550204e+07
1    VIETNAM  5.529509e+07
2      CHINA  5.372449e+07
3      INDIA  5.203551e+07
4      JAPAN  4.541018e+07


An interesting way to look at this data would be to overlay it over a World map using Plotly's dash modules, which AIDA supports internally.

To make it interactive, we can have a slider to select the range of order dates and the possible values of regions provided as radio buttons. A handler function can listen to changes made to either of them and automatically update the data displayed on the map.

In [5]:
def chooseData(dw,app):
    from aida.aida import Q, CMP, C, F, SUM, COL, DATE;
    import dash_core_components as dcc;
    import dash_html_components as html;
    from dash.dependencies import Input, Output, State;
    from datetime import datetime;
    from dateutil.relativedelta import relativedelta;
    import logging;
    
    cmap = {'ALGERIA':'ALA' ,'ARGENTINA':'ARG' ,'BRAZIL':'BRA' ,'CANADA':'CAN' ,'CHINA':'CHN' ,'EGYPT':'EGY' ,'ETHIOPIA':'ETH' ,'FRANCE':'FRA' ,'GERMANY':'DEU' ,'INDIA':'IND' ,'INDONESIA':'IDN' ,'IRAN':'IRN' ,'IRAQ':'IRQ' ,'JAPAN':'JPN' ,'JORDAN':'JOR' ,'KENYA':'KEN' ,'MOROCCO':'MAR' ,'MOZAMBIQUE':'MOZ' ,'PERU':'PER' ,'ROMANIA':'ROU' ,'RUSSIA':'RUS' ,'SAUDI ARABIA':'SAU' ,'UNITED KINGDOM':'GBR' ,'UNITED STATES':'USA' ,'VIETNAM':'VNM'};
    
    def unix_time(dt):
        return (dt - datetime.utcfromtimestamp(0)).total_seconds();
    
    def getMarks(start, end):
        result = []; current = start;
        while current <= end:
            result.append(current);
            current += relativedelta(months=6);
        return {int(unix_time(m)):(str(m.strftime('%Y-%m'))) for m in result};
    
    # Find the date ranges available from the database.
    minDate = datetime.strptime(dw.orders.min(('o_orderdate',)), '%Y-%m-%d');
    maxDate = datetime.strptime(dw.orders.max(('o_orderdate',)), '%Y-%m-%d');
    
    # Find the regions from the database.
    regions = dw.region.project(('r_name',)).cdata['r_name'];
            
    layout = html.Div([ # ==> This is the layout of our interactive widget.
       html.Div(id=dw.genDivId('worldMap')) # ==> We plot the graph here.
       ,dcc.RadioItems(id=dw.genDivId('region') # ==> A set of radio buttons for region
                       ,options=[{'label':r, 'value':r} for r in regions]
                      ,value=regions[0], labelStyle={'display': 'inline-block'})
        #And a slider here.
       ,html.Div([dcc.RangeSlider(id=dw.genDivId('orderDateRange'), min=minDate.timestamp(), max=maxDate.timestamp()
                      ,value=[minDate.timestamp(), maxDate.timestamp()], step=86400
                      ,marks=getMarks(minDate, maxDate))],style={'width':'95%'})
       ]);
    logging.info("created layout");

    def recompute(start, end, regionName): #Rebuild the data based on the range chosen.
        c = dw.customer;l = dw.lineitem; s = dw.supplier; n = dw.nation; 
        r = dw.region.filter(Q('r_name', C(regionName)));
        o = dw.orders.filter(Q('o_orderdate', DATE(start), CMP.GTE), Q('o_orderdate', DATE(end), CMP.LT));

        t = c.join(o, ('c_custkey',),('o_custkey',), COL.ALL, COL.ALL);
        t = t.join(l, ('o_orderkey',), ('l_orderkey',), COL.ALL, COL.ALL);
        t = t.join(s, ('l_suppkey', 'c_nationkey'), ('s_suppkey', 's_nationkey'), COL.ALL, COL.ALL);
        t = t.join(n, ('s_nationkey',), ('n_nationkey',), COL.ALL, COL.ALL);
        t = t.join(r, ('n_regionkey',), ('r_regionkey',), COL.ALL, COL.ALL);
        t = t.project(('n_name', {F('l_extendedprice')*(1-F('l_discount')):'rev'}))
        t = t.aggregate(('n_name', {SUM('rev'):'revenue'}), ('n_name',));
        t = t.order('revenue#desc');

        dw.selData =  t;
        recs = t.cdata;
        logging.info("generating graph");
        return dcc.Graph(figure={ 'data': [ { 'z': recs['revenue']
                                             ,'autocolorscale': False, 'colorbar': {'title': 'Revenue'}
                                             ,'locations': [cmap[c] for c in recs['n_name']]
                                             ,'marker': {'line': { 'color': 'rgb(180,180,180)', 'width': 0.5 }}
                                             ,'reversescale': False, 'type': 'choropleth', 'zmin': 0 } ]
                                 ,'layout': {
                                     "geo": { "projection": {"type": "Mercator"}
                                             ,"showcoastlines": True, "showframe": False }
                                             ,"title": "TPC-H Q5 - Revenue Volumes"
                                              } });

    # This function will be called everytime the slider is adjusted.
    @app.callback( Output(dw.getDivId('worldMap'), 'children')
                  ,[Input(dw.getDivId('orderDateRange'), 'value') ,Input(dw.getDivId('region'), 'value')])
    def updateGraph(ordRange, regionName):
        logging.info("callback activated");
        return recompute(datetime.fromtimestamp(ordRange[0]).strftime('%Y-%m-%d')
                        ,datetime.fromtimestamp(ordRange[1]).strftime('%Y-%m-%d')
                        ,regionName);

    return layout;


Ask AIDA to build and deploy this widget

In [6]:
pltp = dw._Page(chooseData);
show(pltp, height=600);

User can do selections on the interactive widget which will change the information overlayed on the map.

In [7]:
head(dw.selData);

       n_name       revenue
0  MOZAMBIQUE  3.709886e+08
1     ALGERIA  3.653029e+08
2    ETHIOPIA  3.351155e+08
3     MOROCCO  3.213111e+08
4       KENYA  3.112156e+08


In [8]:
dw._close();
dw=None;